Library

In [4]:
# from pyvi import ViTokenizer, ViPosTagger
from utils.transformer import tokenizer

import numpy as np
import pandas as pd
# import gensim
import sklearn
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import *
from keras.layers import *

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.optimizer_v2.adam import Adam
from tensorflow.keras.utils import to_categorical
from configs import ROOT_DIR


ModuleNotFoundError: No module named 'utils'

Data Preparation

In [14]:
# datanewscontent=pd.read_excel(r'path_to_newscontent.xlsx','Sheet1')
dataset_file_path = ROOT_DIR + '\datamining\data\shopee\data.xlsx'
# dataset_file_path = ROOT_DIR + '\datamining\data\shopee\comments.xlsx'
datacomment=pd.read_excel(dataset_file_path,'Sheet1')


In [30]:
datacmt=[]
labelcmt=datacomment['label']
new_labelcmt = []
i=0
a=0
b=0
c=0
for d in datacomment['comment']:
    if labelcmt[i] == 1 :
        new_labelcmt.append(0)
        e=tokenizer(str(d))
        datacmt.append(e)
        a+=1
    if labelcmt[i] == 3 :
        new_labelcmt.append(1)
        e=tokenizer(str(d))
        datacmt.append(e)
        b+=1
    if labelcmt[i] == 5 :
        new_labelcmt.append(2)
        e=tokenizer(str(d))
        datacmt.append(e)
        c+=1
    i+=1
print(a)
print(b)
print(c)

print(len(datacmt))
print(len(labelcmt))

4114
2935
1999
9048
15030


In [8]:
# datanews=[]
# for d in datanewscontent['all_lower']:
#     e=ViTokenizer.tokenize(str(d))
#     datanews.append(e)
# labelnews=datanewscontent['label']

In [23]:
def truncatedvectors(data,n_components=300):
  svd_ngram = TruncatedSVD(n_components=n_components, random_state=42)
  svd_ngram.fit(data)
  return svd_ngram.transform(data)

Word2Vec

In [0]:
from gensim.models import Word2Vec,KeyedVectors 
import os
word2vec_model_path ="path_to_wikivimodel.bin"
model = KeyedVectors.load_word2vec_format(word2vec_model_path,binary=True, unicode_errors='ignore')
vocab = model.wv.vocab
wv = model.wv

In [7]:
def get_word2vec_data(X):
    word2vec_data = []
    for x in X:
        sentence = []
        for word in x.split(" "):
            if word in vocab:
              sentence=sentence+wv[word].ravel().tolist()
        word2vec_data.append(sentence)

    return word2vec_data
def change_to_word2vec(data):
  data2vec=get_word2vec_data(data)
  lengthOfdata=[len(data2vec[i]) for i,n in enumerate(data2vec)]
  for i,n in enumerate(data):
    if(len(data2vec[i])<max(lengthOfdata)):
      for j in range(1,(max(lengthOfdata)-len(data2vec[i]))+1):
        data2vec[i].append(0)
  return truncatedvectors(np.array(data2vec))

In [0]:
X_data_w2v_cmt=change_to_word2vec(datacmt)
X_data_w2v_news=change_to_word2vec(datanews)

TF-IDF

In [24]:
def tfidf(data):
  tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(1, 2))
  tfidf_vect_ngram.fit(data)
  X_data_tfidf_ngram =  tfidf_vect_ngram.transform(data)
  return truncatedvectors(X_data_tfidf_ngram)

In [25]:
X_data_tfidf_cmt=tfidf(datacmt)
# X_data_tfidf_news=tfidf(datanews)

Bag of words

In [0]:
def bow(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(dataall)
  datacmtbow = tokenizer.texts_to_sequences(data)
  datacmtbow= pad_sequences(datacmtbow, maxlen=300)
  return datacmtbow

In [0]:
X_data_bow_cmt=bow(datacmt)
X_data_bow_news=bow(datanews)

LSTM Model

In [26]:
#Change: [X_data_tfidf_cmt,  X_data_w2v_cmt, X_data_bow_cmt],labelcmt , [X_data_tfidf_news,X_data_bow_news,X_data_w2v_news], labelnews

X_train, X_val, y_train, y_val = train_test_split(X_data_tfidf_cmt, new_labelcmt, test_size=0.2, random_state=42)
t = []
for y in y_val:
    if not y in t :
        t.append(y)
print(t)

[1, 0, 2]


In [27]:
#learning_rate: [1e-5,5e-5,1e-4,5e-4,1e-3,5e-3]
opt = Adam(lr=0.001)
def create_lstm_model():
    input_layer = Input(shape=(300,))
    layer = Reshape((10, 30,))(input_layer)
    layer = LSTM(128, activation='relu',dropout=0.5, recurrent_dropout=0.5)(layer)
    layer = Dense(64, activation='relu')(layer)
    layer = Dense(32, activation='relu')(layer)
    output_layer = Dense(3, activation='softmax')(layer)
    classifier = Model(input_layer, output_layer)
    classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return classifier

c:\Users\huanc\Desktop\Ky2Nam4\Duan\CommentRate\venv\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:357: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [28]:
y_train_lstm_encode = to_categorical(y_train)
print(y_train_lstm_encode[1000])
y_val_lstm_encode=to_categorical(y_val)
classifier = create_lstm_model()
classifier.fit(X_train, y_train_lstm_encode, validation_data=(X_val, y_val_lstm_encode), epochs=200, batch_size=32)
#change different epoch and batch_size

ss: 0.6460 - val_accuracy: 0.7055
Epoch 64/200
227/227 [==============================] - 6s 26ms/step - loss: 0.6693 - accuracy: 0.6879 - val_loss: 0.6465 - val_accuracy: 0.7066
Epoch 65/200
227/227 [==============================] - 6s 25ms/step - loss: 0.6660 - accuracy: 0.6877 - val_loss: 0.6405 - val_accuracy: 0.6989
Epoch 66/200
227/227 [==============================] - 5s 24ms/step - loss: 0.6693 - accuracy: 0.6890 - val_loss: 0.6466 - val_accuracy: 0.7028
Epoch 67/200
227/227 [==============================] - 6s 25ms/step - loss: 0.6614 - accuracy: 0.7016 - val_loss: 0.6468 - val_accuracy: 0.6967
Epoch 68/200
227/227 [==============================] - 6s 24ms/step - loss: 0.6687 - accuracy: 0.6989 - val_loss: 0.6395 - val_accuracy: 0.6989
Epoch 69/200
227/227 [==============================] - 5s 24ms/step - loss: 0.6758 - accuracy: 0.6921 - val_loss: 0.6626 - val_accuracy: 0.6956
Epoch 70/200
227/227 [==============================] - 6s 25ms/step - loss: 0.6563 - accuracy: 

In [29]:
y_pred = classifier.predict(X_val, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(y_val, y_pred_bool))

57/57 [==============================] - 3s 5ms/step
              precision    recall  f1-score   support

           0       0.76      0.75      0.76       828
           1       0.66      0.54      0.59       592
           2       0.67      0.88      0.76       390

    accuracy                           0.71      1810
   macro avg       0.70      0.72      0.70      1810
weighted avg       0.71      0.71      0.70      1810

